[this doc on github](https://github.com/dotnet/interactive/tree/master/samples/notebooks/csharp/Docs)

# Variable Sharing

.NET Interactive enables you to write code in multiple languages within a single notebook and in order to take advantage of those languages' different strengths, you might find it useful to share data between them. By default, .NET Interactive provides [subkernels](https://github.com/dotnet/interactive/blob/master/docs/kernels-overview.md) for three different languages within the same process. You can share variables between .NET subkernels using the `#!share` magic command.

In [6]:
#!csharp
var x = "C# subkernel value";
x

C# subkernel value

In [7]:
#!fsharp
#!share --from csharp x
sprintf "%s, accessed from F# subkernel" x

C# subkernel value, accessed from F# subkernel

Variables are shared by reference for reference types. A consequence of this is that if you share a mutable object, changes to its state will be visible across subkernels:

In [8]:
#!fsharp
open System.Collections.Generic

let list = List<string>()
list.Add "Added by F#"
list    

index,value
0,Added by F#


In [9]:
#!csharp
#!share --from fsharp list

list.Add("Added by C#");

list

index,value
0,Added by F#
1,Added by C#


In [10]:
#!fsharp

list

index,value
0,Added by F#
1,Added by C#


# Direct data entry with `#!value`

It's common to have text that you'd like to use in a notebook. It might be JSON, CSV, XML, or some other format. It might be in a file, in your clipboard, or on the web. The `#!value` magic command is available to make it as easy as possible to get that text into a variable in your notebook. An important thing to know is that `#!value` is an alias to a  subkernel designed just to hold values. This means that once you store something in it, you can access it from another subkernel using `#!share`.

There are three ways to use `#!value` to get data into your notebook session:

### 1. From the clipboard

 The simplest way to use `#!value` is to paste some text into the cell. The text will be stored as a string, but unlike using a `string` literal in C#, F#, or PowerShell, there's no need to escape anything.


In [11]:
#!value --name someJson
    
{ 
    "what": "some JSON", 
    "why": "to share it with another subkernel"
}

In [12]:
#!share someJson --from value
    
using Newtonsoft.Json.Linq;
var jObject = JObject.Parse(someJson);
jObject.ToString()

{
  "what": "some JSON",
  "why": "to share it with another subkernel"
}


### 2. From a file

If the data you want to read into your notebook is stored in a file, you can use `#!value` with the `--from-file` option:

In [13]:
#!value --from-file data.json --name someJson

In [14]:
#!share someJson --from value
someJson

{
    "what": "widget",
    "howMany": 123
}


### 3. From a URL

You can pull data into your notebook from a URL as well, using the `--from-url` option. 


In [15]:
#!value --from-url https://dot.net --name dn

In [ ]:
#!share --from value dn
display(dn, "text/html");

## Specifying a MIME type

Regardless of which of these approaches you use, you can additionally choose to display the value in the notebook at the time of submission by using the `--mime-type` option. This accomplishes a few things. If your notebook frontend knows how to display that mime type, you can see it appropriately formatted:

In [17]:
#!value --name someJson --mime-type application/json
    
{ 
    "what": "some JSON", 
    "why": 
        [
            "to share it with another subkernel",
            "to see it in a treeview"
        ]
}

This also causes the value to be saved in your `.ipynb` file, something that would not otherwise happen.



This also effectively stores the value in your `.ipynb` file, something that would not otherwise happen.


## Limitations

Variable sharing has some limitations to be aware of. When sharing a variable with a subkernel where its compilation requirements aren't met, for example due to a missing `using` (C#) or `open` (F#) declaration, a custom type defined in the notebook, or a missing assembly reference, `#!share` will fail. This limitation may be lifted in the future but for now, if you want to share variables of types that aren't imported by default, you will have to explicitly run the necessary import code in the destination kernel.


In [18]:
#!csharp 
public class DefinedInCSharp { }

var csharpInstance = new DefinedInCSharp();

In [19]:
#!fsharp
#!share --from csharp csharpInstance
csharpInstance

Unhandled exception: System.InvalidOperationException: Unable to import type, Submission#17+DefinedInCSharp.
 ---> System.InvalidOperationException: Unable to import type, Submission#17+DefinedInCSharp.
   at FSharp.Compiler.Interactive.Shell.FsiDynamicCompiler.importReflectionType(FsiDynamicCompilerState istate, Type reflectionTy) in F:\workspace\_work\1\s\src\fsharp\fsi\fsi.fs:line 1340
   at FSharp.Compiler.Interactive.Shell.FsiDynamicCompiler.AddBoundValue(CompilationThreadToken ctok, ErrorLogger errorLogger, FsiDynamicCompilerState istate, String name, Object value) in F:\workspace\_work\1\s\src\fsharp\fsi\fsi.fs:line 1595
   --- End of inner exception stack trace ---
   at FSharp.Compiler.Interactive.Shell.FsiEvaluationSession.commitResult[a,b](FSharpChoice`2 res) in F:\workspace\_work\1\s\src\fsharp\fsi\fsi.fs:line 2831
   at FSharp.Compiler.Interactive.Shell.FsiEvaluationSession.AddBoundValue(String name, Object value) in F:\workspace\_work\1\s\src\fsharp\fsi\fsi.fs:line 